In [2]:
import gc

import dash
import pandas as pd
import plotly.express as px
import yaml
from dash import dcc, html
from dash.dependencies import Input, Output

from mongodb_lib import *

config_infra = yaml.load(open("../infra-config-pipeline.yaml"), Loader=yaml.FullLoader)
db, fs, client = connect_to_mongodb(config_infra)

INFO:root:Successfully connected to MongoDB.


In [5]:
df = read_object(fs, "product_textual_english_summarized")
df = pd.DataFrame(df)

In [16]:
prd_id="129335P2"
print("Title: " + list(df[df["PRODUCTCODE"]==prd_id]["pdt_product_detail_PRODUCTTITLE_translated"])[0])
print("Description: " + list(df[df["PRODUCTCODE"]==prd_id]["pdt_product_detail_PRODUCTDESCRIPTION_SUMMARIZED"])[0])

Title: Montmartre/Sacre-Coeur/Paris- Artistic & Bohemian Epicenter 
Description: Walking tour covers must-see spot as well as hidden treasures of Montmartre Village. Walking in the footsteps of the impressionist artists who were inspired by local sites. You see film shot spots of famous French ‘ Amelie of MontMartre’ and many more.


In [17]:
lll = ['366458P5', '416844P1', '5944P8']

for el in lll:

    print("Title: " + list(df[df["PRODUCTCODE"]==el]["pdt_product_detail_PRODUCTTITLE_translated"])[0])
    print("Description: " + list(df[df["PRODUCTCODE"]==el]["pdt_product_detail_PRODUCTDESCRIPTION_SUMMARIZED"])[0])

Title: Montmarte and Sacré-Coeur with the best guides in Paris
Description: Explore the picturesque alleys of Montmartre. Stroll along the rue des Abbesses, witness to an ancient history. Admire the breathtaking panoramic view of Paris.
Title: Bohemian Montmartre Private Walking Tour 
Description: The Moulin Rouge and Sacré Coeur are just a few of the sights to explore. The tour normally starts at 9.30am but I can be flexible, please message me. I can also adapt to your timing.
Title: Discover Paris: Hidden Montmartre Tour
Description: Explore the cobbled streets of Montmartre, the fabled 2,000-year-old district. Taste one of the best butter croissants in all of Paris. Discover the Sacre-Coeur Basilica from a different angle.
